In [10]:
!pip3 install yfinance scikit-learn stable-baselines3
!apt-get install python-opengl -y python3.8-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following additional packages will be installed:
  libpython3.8 libpython3.8-dev zlib1g-dev
The following NEW packages will be installed:
  libpython3.8 libpython3.8-dev python3.8-dev zlib1g-dev
0 upgraded, 4 newly installed, 0 to remove and 0 not upgraded.
Need to get 56.7 MB of archives.
After this operation, 97.5 MB of additional disk space will be used.
Get:1 http://ubuntu.mirrors.ovh.net/ubuntu bionic-updates/universe amd64 libpython3.8 amd64 3.8.0-3~18.04.1 [1632 kB]
Get:2 http://ubuntu.mirrors.ovh.net/ubuntu bionic-updates/universe amd64 libpython3.8-dev amd64 3.8.0-3~18.04.1 [54.4 MB]
Get:3 http://ubuntu.mirrors.ovh.net/ubuntu bionic/main amd64 zlib1g-dev amd64 1:1.2.11.dfsg-0ubuntu2 [176 kB]
Get:4 http://ubuntu.mirrors.ovh.net/ubuntu bionic-updates/universe amd64 python3.8-dev amd64 3.8.0-3~18.04.1 [510 kB]
Fetched 56.7

In [11]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz && tar -xzf ta-lib-0.4.0-src.tar.gz && cd ta-lib/ && ./configure --prefix=/usr && make && sudo make install

--2021-06-10 11:17:29--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-10 11:17:29--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://newcontinuum.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-10 11:17:29--  https://newcontinuum.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving newcon

In [12]:
!pip3 install TA-Lib

  Using cached TA-Lib-0.4.20.tar.gz (266 kB)
  Created wheel for TA-Lib: filename=TA_Lib-0.4.20-cp38-cp38-linux_x86_64.whl size=1766763 sha256=314a6119240f312c75741eea65762875e8765b2f0df7dbde3a67a84285165a52
  Stored in directory: /root/.cache/pip/wheels/4f/b1/e4/632425cebc7de5ceee7f6d3a373a717764038d199811eec544
Successfully built TA-Lib


In [13]:
import time
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
import gym
from gym import spaces
from gym.utils import seeding
from gym.envs.classic_control import rendering
import talib as ta

In [14]:
MONKEY_HIGH = 1
NUMBER_OF_ROPES = 4
DoGym = False

In [15]:
class monkeyEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, monkey_high, n):

        self.monkey_high = monkey_high
        self.viewer = None
        self.n = n+1

        # actions
        self.action_space = spaces.Discrete(self.n)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(1,),
            dtype=np.float32)

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self):
        self.monkey_last_pos = 0
        self.monkey_pos = 0
        self.rope_high = 1
        self.last_rope_high = 1
        self.monkey_last_high = MONKEY_HIGH
        self.monkey_high = MONKEY_HIGH
        self.last_time = 0
        self.time = 0

        self.ropes = pd.DataFrame(
            np.random.randint(
                1, 100, size=(100, NUMBER_OF_ROPES)
            ),
            columns=list(range(1, NUMBER_OF_ROPES+1))
        )
        # let assume that is an stable coin like usdt that have 50$ price
        self.ropes.insert(loc=0, column=0, value=[50]*100)

        # for idx, rope in enumerate(self.ropes):
        #     self.ropes[idx] = ta.SMA(rope, 14)
        if self.viewer != None:
            self.viewer.close()
            self.viewer = None
        # print(self.ropes.head())

        return np.array([self.monkey_pos]).astype(np.float32)

    def step(self, action):

        self.monkey_last_pos = self.monkey_pos
        self.monkey_pos = action

        # if self.monkey_last_pos != self.monkey_pos:
        self.monkey_last_high = self.monkey_high
        self.monkey_high = self.ropes.iat[self.time, self.monkey_pos]

        self.time += 1
        done = True if self.time == 100 else False
        reward = 0.01*(self.monkey_high - self.monkey_last_high)

        info = {
            "monkey_last_pos": self.monkey_last_pos,
            "monkey_pos": self.monkey_pos,

            "rope_high": self.rope_high,
            "last_rope_high": self.last_rope_high,

            "monkey_last_high": self.monkey_last_high,
            "monkey_high": self.monkey_high,

            "last_time": self.last_time,
            "time": self.time,

        }

        return np.array([self.monkey_pos]).astype(np.float32), reward, done, info

    def render(self, mode='human'):

        x = self.time - 1
        y = self.monkey_high
        if self.viewer is None:

            self.viewer = rendering.Viewer(2000, 500)

            for i in range(self.n):
                xs = pd.Series(range(100))
                ys = self.ropes.iloc[:, i]
                xys = list(zip(xs*20, ys*5))
                self.track = rendering.make_polyline(xys)
                self.track.set_linewidth(2)
                self.track.set_color(*np.random.rand(3))
                self.viewer.add_geom(self.track)

            self.monkey = rendering.make_circle(radius=5)

            self.monkey.set_color(0, 0, 0)
            self.monkeyT = rendering.Transform(translation=(x*20, y*5))
            self.monkey.add_attr(self.monkeyT)
            self.viewer.add_geom(self.monkey)
        else:
            self.monkeyT.set_translation(x*20, y*5)

        return self.viewer.render(return_rgb_array=mode == 'rgb_array')

    def close(self):
        pass

In [16]:
symbols = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD']
df = pd.DataFrame([1]*100)

for i in range(len(symbols)):
    symbol = symbols[i]
    s = yf.download(tickers=symbol, period='22h',
                    interval='15m')['Close']
    print(s)
    df.insert(loc=i+1, column=i+1, value=s)

scaler = MinMaxScaler()

x = df.values  # returns a numpy array
x_scaled = scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

[*********************100%***********************]  1 of 1 completed
Datetime
2021-06-09 15:15:00+01:00    34934.550781
2021-06-09 15:30:00+01:00    35167.695312
2021-06-09 15:45:00+01:00    34715.722656
2021-06-09 16:00:00+01:00    35748.468750
2021-06-09 16:15:00+01:00    36398.976562
                                 ...     
2021-06-10 11:30:00+01:00    38205.777344
2021-06-10 11:45:00+01:00    37893.468750
2021-06-10 12:00:00+01:00    38051.117188
2021-06-10 12:15:00+01:00    38019.328125
2021-06-10 12:17:02+01:00    38017.214844
Name: Close, Length: 86, dtype: float64
[*********************100%***********************]  1 of 1 completed
Datetime
2021-06-09 15:15:00+01:00    2513.297363
2021-06-09 15:30:00+01:00    2523.149414
2021-06-09 15:45:00+01:00    2464.733643
2021-06-09 16:00:00+01:00    2514.794678
2021-06-09 16:15:00+01:00    2554.099854
                                ...     
2021-06-10 11:30:00+01:00    2594.710938
2021-06-10 11:45:00+01:00    2579.125488
2021-06-10 12:

/root/venv/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/root/venv/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [20]:
env = monkeyEnv(monkey_high=MONKEY_HIGH, n=NUMBER_OF_ROPES)
model = None
DoGym=True
if DoGym:
    model = PPO("MlpPolicy", env, verbose=1)
    model.learn(total_timesteps=100000)
    model.save("ppo_monkey")

# del model  # remove to demonstrate saving and loading

# model = PPO.load("ppo_monkey")

obs = env.reset()
while True:
    time.sleep(0.1)
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
    print(info['monkey_high'], info['monkey_pos'])
    if info['time'] >= 100:
        env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | 0.52        |
| time/                   |             |
|    fps                  | 2463        |
|    iterations           | 1           |
|    time_elapsed         | 0           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.003655871 |
|    clip_fraction        | 0.0165      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.885      |
|    explained_variance   | -0.00537    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0157      |
|    n_updates            | 490         |
|    policy_gradient_loss | -0.00119    |
|    value_loss           | 0.026       |
-----------------------------------------
------------------------

KeyboardInterrupt: 